In [ ]:
# import zipfile
# import os

# # 2. Распаковка в папку pdf_files
# zip_filename = "pdf_files.zip"
# extract_folder = "pdf_files"

# # Создаём папку, если её нет
# os.makedirs(extract_folder, exist_ok=True)

# # Распаковываем
# with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
#     zip_ref.extractall(extract_folder)

# print(f"Архив {zip_filename} распакован в папку {extract_folder}/")

!unzip -q pdf_files.zip -d /content  # Распаковываем в корень
!mv /content/pdf_files /content/my_pdfs  # Переименовываем (опционально)

In [ ]:
import os
import re
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.neighbors import NearestNeighbors
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder

In [ ]:
import os
import re
import fitz
from PIL import Image, UnidentifiedImageError
import pandas as pd

def extract_images_from_pdf(pdf_path, output_folder):
    doc = fitz.open(pdf_path)
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    metadata_list = []
    os.makedirs(output_folder, exist_ok=True)

    # Получаем оглавление (TOC) для определения глав
    toc = doc.get_toc()

    # Улучшенное регулярное выражение для подписей
    caption_pattern = re.compile(
        r'^(?:(?:Fig(?:ure)?|Рис(?:унок)?)\s*[\d\.]+[\-–:]?|'  # Английские и русские варианты
        r'^\d+[\.\-–]?\s*|'                                   # Нумерация в начале строки
        r'^[A-Za-zА-Яа-я]+\s*\d+[\.\-–:]?)'                   # Текстовые префиксы
        r'\s*(.*)', 
        re.IGNORECASE | re.MULTILINE
    )

    for page_num in range(len(doc)):
        page = doc[page_num]
        images = page.get_images(full=True)
        text_blocks = page.get_text("blocks")

        # Определяем главу для текущей страницы
        chapter = None
        for level, title, p in toc:
            if p <= page_num + 1:
                chapter = title
            else:
                break

        for img_idx, img in enumerate(images):
            xref = img[0]
            try:
                base_image = doc.extract_image(xref)
                img_data = base_image["image"]
                img_ext = base_image["ext"]
                img_name = f"{base_name}_p{page_num+1}_i{img_idx+1}.{img_ext}"
                img_path = os.path.join(output_folder, img_name)
                
                with open(img_path, "wb") as f:
                    f.write(img_data)

                pil_img = Image.open(img_path)
                width, height = pil_img.size
                is_color = 1 if pil_img.mode in ['RGB', 'RGBA'] else 0

                # Определение класса (agr/ds)
                file_class = 'agr' if 'agr' in base_name.lower() else 'ds' if 'ds' in base_name.lower() else None

                # Поиск подписи в радиусе 100 пунктов вокруг изображения
                caption = None
                try:
                    img_rect = page.get_image_bbox(xref)
                    search_rect = img_rect + (-50, -50, 50, 100)  # Расширенная область поиска
                    
                    # Собираем весь текст вокруг изображения
                    context_text = page.get_text("text", clip=search_rect)
                    
                    # Ищем подпись в собранном тексте
                    for line in context_text.split('\n'):
                        line = line.strip()
                        if match := caption_pattern.match(line):
                            caption = match.group(1).strip()
                            if not caption:  # Если группа пустая, берем всю строку
                                caption = line
                            break
                            
                except Exception as e:
                    print(f"Ошибка при поиске подписи: {e}")

                metadata_list.append({
                    "image_path": img_path,
                    "source_file": base_name,
                    "page": page_num + 1,
                    "chapter": chapter,
                    "width": width,
                    "height": height,
                    "is_color": is_color,
                    "class": file_class,
                    "caption": caption
                })

            except (ValueError, UnidentifiedImageError, fitz.fitz.BaseError) as e:
                print(f"Ошибка при обработке изображения {xref} на странице {page_num + 1}: {e}")
                continue

    doc.close()
    return metadata_list

if __name__ == '__main__':
    pdf_folder = "pdf_files"
    output_root = "extracted_images_3"
    all_metadata = []

    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.lower().endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            pdf_output_folder = os.path.join(output_root, os.path.splitext(pdf_file)[0])
            try:
                print(f"Обработка файла: {pdf_file}...")
                metadata = extract_images_from_pdf(pdf_path, pdf_output_folder)
                all_metadata.extend(metadata)
                print(f"Файл {pdf_file} успешно обработан. Извлечено изображений: {len(metadata)}")
            except fitz.fitz.FileNotFoundError:
                print(f"ОШИБКА: Файл не найден: {pdf_file}")
            except fitz.fitz.EmptyFileError:
                print(f"ОШИБКА: Пустой PDF файл: {pdf_file}")
            except fitz.fitz.PDFError as e:
                print(f"ОШИБКА: Поврежденный PDF файл {pdf_file}: {str(e)}")
            except PermissionError:
                print(f"ОШИБКА: Нет доступа к файлу: {pdf_file}")
            except Exception as e:
                print(f"КРИТИЧЕСКАЯ ОШИБКА при обработке файла {pdf_file}: {str(e)}")
                print(f"Тип исключения: {type(e).__name__}")
                if hasattr(e, '__traceback__'):
                    import traceback
                    traceback.print_exc()
    # Создаем DataFrame и сохраняем
    df = pd.DataFrame(all_metadata)
    df.to_csv("images_metadata.csv", index=False)

In [ ]:
# 3) Обучение классификатора - 6 классов 

def train_classifier(data_root, epochs=5, bs=32, lr=1e-3, device='cuda'):
    tf = transforms.Compose([
        transforms.Resize((224,224)), transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    ds = ImageFolder(os.path.join(data_root,'train'), transform=tf)
    tr,va = random_split(ds,[int(0.8*len(ds)),len(ds)-int(0.8*len(ds))])
    dl_tr = DataLoader(tr, bs, shuffle=True)
    dl_va = DataLoader(va, bs)

    m = models.resnet50(pretrained=True)
    m.fc = nn.Linear(m.fc.in_features, len(ds.classes))
    m = m.to(device)
    opt = optim.Adam(m.parameters(), lr=lr)
    crit = nn.CrossEntropyLoss()
    for e in range(epochs):
        m.train()
        tot=0
        for x,y in dl_tr:
            x,y = x.to(device),y.to(device)
            opt.zero_grad(); z=m(x); loss=crit(z,y); loss.backward(); opt.step()
            tot+=loss.item()
        m.eval()
        cor=0; cnt=0
        with torch.no_grad():
            for x,y in dl_va:
                x,y=x.to(device),y.to(device)
                cor+= (m(x).argmax(1)==y).sum().item(); cnt+=y.size(0)
        print(f"Ep{e+1} L={tot/len(dl_tr):.3f} Acc={cor/cnt:.3f}")
    return m, ds.classes

# обучение:
# classifier,classes = train_classifier('/mnt/data/data', epochs=3)


# 4) Строим эмбеддинги и поиск похожих

def get_embeddings(paths, backbone, classifier=None, device='cuda'):
    bb = torch.nn.Sequential(*list(backbone.children())[:-1]).to(device).eval()
    tf = transforms.Compose([
        transforms.Resize((224,224)), transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    embs=[]
    preds=[]
    for p in paths:
        im = Image.open(p).convert('RGB')
        x = tf(im).unsqueeze(0).to(device)
        with torch.no_grad():
            e = bb(x).squeeze().cpu().numpy(); embs.append(e)
            if classifier:
                preds.append(classes[classifier(x).argmax(1).item()])
    return np.vstack(embs), preds

class Searcher:
    def __init__(self, embs, paths, df):
        self.nn = NearestNeighbors(metric='cosine').fit(embs)
        self.paths,self.df = paths,df
    def query(self, qpath, k=5):
        e,_ = get_embeddings([qpath], backbone, classifier)
        ds,idxs = self.nn.kneighbors(e, k)
        res=[]
        for d,i in zip(ds[0],idxs[0]):
            row=self.df.iloc[i]
            res.append({'path':row.image_path,'file':row.source_file,'page':row.page,'dist':d})
        return res

# Пример поиска
# backbone = models.resnet50(pretrained=True)
# searcher = Searcher(embs, df.image_path.tolist(), df)\# results = searcher.query('/mnt/data/query.png')